Creating your First Graph and Running it in a Session

In [1]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")

f = x*x*y + y + 2

In [2]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


Global variable initializer

In [3]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


Managing Graph

In [4]:
# Any node that is created is automatically added to the default graph
x1 = tf.Variable(1, name="x1")
x1.graph is tf.get_default_graph()

True

In [5]:
# Managing multiple independent graph
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2, name="x2")
    
x2.graph is graph

True

In [6]:
x2.graph is tf.get_default_graph()

False

Lifecycle of a Node Value

All node values are dropped between graph runs, except variable values which are maintained by the session across graph runs.

In [7]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

# Below code evaluates w & x twice

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


If one wants to evaluate y & z effectively, without evaluating w & x twice, one must evaluate y & z in just one graph run

In [8]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


Linear Regression with TensorFlow

In [9]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape

In [10]:
print(m, n)

20640 8


In [30]:
# this line prevents creation of duplicate nodes
tf.reset_default_graph()

housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

XT = tf.transpose(X)

# theta = ((XT * X) ^ -1) * XT * y

theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    
    print(theta_value)

[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


Gradient Descent

In [31]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(housing.data)

scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaler.transform(housing.data)]
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")


# Name scope
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")
    
print(error.op.name)
print(mse.op.name)

loss/sub
loss/mse


In [32]:
# Manually computing the gradient
# gradients = 2/m * tf.matmul(tf.transpose(X), error)
# training_op = tf.assign(theta, theta - learning_rate * gradients)

# Using autodiff
# gradients = tf.gradients(mse, [theta])[0]
# training_op = tf.assign(theta, theta - learning_rate * gradients)

# Using an optimizer 
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

# Momentum optimizer
# optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
# training_op = optimizer.minimize(mse)

# Saving the model
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")
    
    print(best_theta)

Epoch 0 MSE =  12.953072
Epoch 100 MSE =  0.7638138
Epoch 200 MSE =  0.5937989
Epoch 300 MSE =  0.57281864
Epoch 400 MSE =  0.55940825
Epoch 500 MSE =  0.5497429
Epoch 600 MSE =  0.5427531
Epoch 700 MSE =  0.5376953
Epoch 800 MSE =  0.5340338
Epoch 900 MSE =  0.53138185
[[ 2.0685525e+00]
 [ 7.6767164e-01]
 [ 1.3453381e-01]
 [-9.7243950e-02]
 [ 1.4400561e-01]
 [ 2.0423972e-03]
 [-3.9340470e-02]
 [-8.4301001e-01]
 [-8.0397642e-01]]


Feeding Data to the Training Algorithm

Placeholder node

In [25]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8 , 9]]})
    
print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


Mini-Batch Gradient Descent

In [34]:
# Visualizing the Graph and Training Curves using TensorBoard
# tensorboard --logdir tf_logs/
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}".format(root_logdir, now)

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

X_data = scaled_housing_data_plus_bias.astype(np.float32)
y_data = housing.target.reshape(-1, 1).astype(np.float32)

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

n_epochs = 1000
batch_size = 100
n_bacthes = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    start_index = batch_index * batch_size
    end_index = (batch_index + 1) * batch_size
    
    return X_data[start_index: end_index], y_data[start_index: end_index]

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        for batch_index in range(n_bacthes):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X:X_batch, y:y_batch})
                step = epoch * n_bacthes + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
            
    best_theta = theta.eval()
    print(best_theta)

file_writer.close()

Epoch 0 MSE =  9.037816
Epoch 100 MSE =  0.5243211
Epoch 200 MSE =  0.5243211
Epoch 300 MSE =  0.5243211
Epoch 400 MSE =  0.5243211
Epoch 500 MSE =  0.5243211
Epoch 600 MSE =  0.5243211
Epoch 700 MSE =  0.5243211
Epoch 800 MSE =  0.5243211
Epoch 900 MSE =  0.5243211
[[ 2.0685525 ]
 [ 0.82963324]
 [ 0.11875443]
 [-0.26555082]
 [ 0.3057157 ]
 [-0.00450222]
 [-0.03932687]
 [-0.8998529 ]
 [-0.8705097 ]]



Restoring Model

In [24]:
with tf.Session() as sess:
    saver.restore(sess, '/tmp/my_model_final.ckpt')
    var_names = [x.name for x in saver._var_list]
    print(var_names)
    #theta = tf.get_default_graph().get_tensor_by_name('theta:0')
    #momentum_theta = tf.get_default_graph().get_tensor_by_name('theta/Momentum_2:0')
    print(theta.eval())
    #print(momentum_theta.eval())
    #best_theta = theta.eval()
    #print(best_theta)

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt
['theta:0', 'theta/Momentum:0', 'theta/Momentum_1:0']
[[ 2.0685582 ]
 [ 0.82963306]
 [ 0.11875416]
 [-0.2655526 ]
 [ 0.30571756]
 [-0.00450227]
 [-0.03932677]
 [-0.89985454]
 [-0.8705114 ]]


To-Do 
- Do batch gradient descent
- Saving and storing tensorflow model

TensorBoard